<a href="https://colab.research.google.com/github/kdy5213/AI-ExpertFW-Prj/blob/main/cloth_recommendation_ML2_KDY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###머신러닝 관련 메모

In [ ]:

머신러닝 범주형 데이터 처리
https://blog.naver.com/PostView.naver?blogId=dalgoon02121&logNo=222088976883&parentCategoryNo=&categoryNo=37&viewDate=&isShowPopularPosts=true&from=search

- 문자열(범주형) 값을 내림차순 정렬 후 0부터 1씩 증가하는 값으로 변환함.

- 숫자의 차이가 모델에 영향을 주지 않는 트리 계열 모델에 적용함.

    (의사결정나무, 랜덤포레스트)
결정 트리와 랜덤 포레스트는 특성 스케일 조정에 대해 걱정할 필요가 없는 몇 안되는 머신러닝 알고리즘 중 하나이다.
대부분 머신러닝과 최적화 알고리즘은 특성의 스케일이 같을때 성능이 훨씬 좋다

- 숫자의 차이가 모델에 영향을 미치는 선형 계열 모델에는 적용하지 않는 것이 좋음.

    (로지스틱 회귀, SVM, 신경망 → 원핫인코딩 적용)


Up sampling : HH2_2022_2.ipynb 참고
sm = SMOTE(sampling_strategy = 1 ,k_neighbors = 5, random_state=1)   
#Synthetic Minority Over Sampling Technique
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())
Down Sampling : HH2_2022_2.ipynb 참고


sklearn.preprocessing.LabelEncoder 활용 방법

> fit() : 어떻게 변환할 것인지에 대해 학습

> transform() : 문자열을 숫자로 변환

> fit_transform() : 학습과 변환을 한 번에 처리

> inverse_transform() : 숫자를 다시 문자열로 변환

> classes_ : 인코딩한 클래스 조회


벨 인코딩에도 뚜렷한 장점이 있는데 첫째, 컬럼의 수가 1개로 줄어서 메모리의 관점에서 매우 효율적입니다. 
둘째, 변수가 순서를 가지는 경우, 예를 들어서 수능 1등급 집단, 2등급 집단, 3등급 집단 ... 
혹은 10대, 20대, ... , 90대와 같이 순서 자체가 진짜 의미가 있는 경우에는 라벨 인코딩이 효과적일 수 있습니다. 


train_test_split(..., stratify = y, random_state = 1)
stratify 값을 target으로 지정해주면 각각의 class 비율(ratio)을 train / validation에 유지해줍니다.(https://ysyblog.tistory.com/71)

scaling 할 때는 train data scaling 한 것 기준으로 test data를 scaling



np.unique(y) 함수는 iris.target에 저장된 세 개의 고유한 클래스 레이블을 반환합니다.

정수 레이블을 사용하는 이유는 사소한 실수를 피할 수 있고 작은 메모리 영역을 차지하므로 계산 성능을 향상 시킵니다.

클래스 레이블을 정수로 인코딩하는 것은 대부분 머신러닝 라이브러리들의 공통된 관례이기도 합니다.

처음 본 데이터에서 훈련된 모델 성능을 평가하기 위해 데이터셋을 훈련 세트와 테스트 세트로 분할합니다.(ppt 참고)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


### 전처리 데이터 불러오기

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/GitHub/AI-ExpertFW-Prj/dataset')

In [ ]:
mc_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/GitHub/AI-ExpertFW-Prj/dataset/modcloth.xlsx')
mc_df.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,user_name,length,fit,user_id,shoe size,shoe width,review_summary,review_text
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,Emily,just right,small,991571,NaN,NaN,NaN,NaN
1,123373,31.0,13,3.0,b,30.0,36.0,new,NaN,5ft 2in,sydneybraden2001,just right,small,587883,NaN,NaN,NaN,NaN
2,123373,30.0,7,2.0,b,NaN,32.0,new,NaN,5ft 7in,Ugggh,slightly long,small,395665,9.0,NaN,NaN,NaN
3,123373,NaN,21,5.0,dd/e,NaN,NaN,new,NaN,NaN,alexmeyer626,just right,fit,875643,NaN,NaN,NaN,NaN
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,dberrones1,slightly long,small,944840,NaN,NaN,NaN,NaN


In [ ]:
modcloth_preprocessing3 = pd.read_excel('modcloth_preprocessing3.xlsx')


In [ ]:
modcloth_preprocessing3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62994 entries, 0 to 62993
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   item_id          62994 non-null  int64  
 1   size             62994 non-null  int64  
 2   rating           62994 non-null  int64  
 3   height           62994 non-null  float64
 4   length           62994 non-null  int64  
 5   fit              62994 non-null  int64  
 6   user_id          62994 non-null  int64  
 7   review_summary   58106 non-null  object 
 8   review_text      58106 non-null  object 
 9   cate_bottoms     62994 non-null  int64  
 10  cate_dresses     62994 non-null  int64  
 11  cate_new         62994 non-null  int64  
 12  cate_outerwear   62994 non-null  int64  
 13  cate_sale        62994 non-null  int64  
 14  cate_tops        62994 non-null  int64  
 15  cate_wedding     62994 non-null  int64  
 16  waist            62994 non-null  float64
 17  hips        

In [ ]:
modcloth_preprocessing3.head()

,item_id,size,rating,height,length,fit,user_id,review_summary,review_text,cate_bottoms,...,cate_new,cate_outerwear,cate_sale,cate_tops,cate_wedding,waist,hips,bra_size,bust_size,cup_size_in_cms
0,123373,7,5,167.64,3,1,991571,NaN,NaN,0,...,1,0,0,0,0,29.000,38.000000,34.0,36.000000,18.5
1,123373,13,3,157.48,3,1,587883,NaN,NaN,0,...,1,0,0,0,0,31.000,30.000000,36.0,35.724218,14.5
2,123373,7,2,170.18,4,1,395665,NaN,NaN,0,...,1,0,0,0,0,30.000,34.438637,32.0,32.904880,14.5
3,123373,18,5,157.48,4,1,944840,NaN,NaN,0,...,1,0,0,0,0,31.325,39.688948,36.0,36.370556,14.5
4,123373,11,5,162.56,3,1,162012,NaN,NaN,0,...,1,0,0,0,0,27.000,41.000000,36.0,37.467970,16.5


### 데이터 split

In [ ]:
modcloth_preprocessing3.keys()

Index(['item_id', 'size', 'rating', 'height', 'length', 'fit', 'user_id',
       'review_summary', 'review_text', 'cate_bottoms', 'cate_dresses',
       'cate_new', 'cate_outerwear', 'cate_sale', 'cate_tops', 'cate_wedding',
       'waist', 'hips', 'bra_size', 'bust_size', 'cup_size_in_cms'],
      dtype='object')

In [ ]:
#X_data = modcloth_preprocessing3[(modcloth_preprocessing3['fit'] == 2) & (modcloth_preprocessing3['length'] == 3)][['height', 'waist', 'hips', 'bra_size', 'bust_size', 'cup_size_in_cms', 'cate_bottoms', 'cate_dresses', 'cate_new', 'cate_outerwear', 'cate_sale', 'cate_tops', 'cate_wedding']]

X_data = modcloth_preprocessing3[(modcloth_preprocessing3['fit'] == 2) & (modcloth_preprocessing3['length'] == 3)][['height', 'waist', 'hips', 'bra_size', 'bust_size', 'cup_size_in_cms', 'cate_bottoms', 'cate_dresses', 'cate_new', 'cate_outerwear', 'cate_sale', 'cate_tops', 'cate_wedding']]
Y_data = modcloth_preprocessing3[(modcloth_preprocessing3['fit'] == 2) & (modcloth_preprocessing3['length'] == 3)]['size']

In [ ]:
X_data.head()

,height,waist,hips,bra_size,bust_size,cup_size_in_cms,cate_bottoms,cate_dresses,cate_new,cate_outerwear,cate_sale,cate_tops,cate_wedding
7,167.64,31.800000,41.000000,36.0,39.000000,20.5,0,0,1,0,0,0,0
8,165.10,29.216667,36.884795,34.0,34.339232,16.5,0,0,1,0,0,0,0
9,165.10,28.400000,30.000000,34.0,36.587500,20.5,0,0,1,0,0,0,0
10,160.02,27.850000,36.000000,34.0,34.764329,14.5,0,0,1,0,0,0,0
11,157.48,31.900000,44.000000,36.0,41.733333,18.5,0,0,1,0,0,0,0


In [ ]:
X_data = modcloth_preprocessing3[['size', 'height', 'waist', 'hips', 'bra_size', 'bust_size', 'cup_size_in_cms', 'cate_bottoms', 'cate_dresses', 'cate_new', 'cate_outerwear', 'cate_sale', 'cate_tops', 'cate_wedding']]
Y_data = modcloth_preprocessing3['fit']

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2 # taking 70:30 training and test set
seed = 1  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=test_size, random_state=seed)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

# 교차검증시
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
y_train.value_counts().sort_index()

1     7839
2    35819
3     6737
Name: fit, dtype: int64

### 데이터 resampling(선택사항.. 하면 결과 더 안좋아지는 경우 존재)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = 'auto' ,k_neighbors = 5, random_state=1)   
#Synthetic Minority Over Sampling Technique
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [ ]:
df_y_train_res = pd.DataFrame(y_train_res, columns = ['size'])

df_y_train_res.value_counts().sort_index()

size
0       8293
1       8293
2       8293
3       8293
4       8293
5       8293
6       8293
7       8293
8       8293
9       8293
10      8293
11      8293
12      8293
13      8293
14      8293
15      8293
17      8293
18      8293
dtype: int64

In [ ]:
DMwR::SMOTE(
  form,  # 모델 포뮬러
  data,  # 포뮬러를 적용할 데이터
  perc.over=200,  # 적은 쪽의 데이터를 얼마나 추가로 샘플링해야 하는지
  k=5,            # 고려할 최근접 이웃의 수
  # 적은 쪽의 데이터를 추가로 샘플링할 때 각 샘플에 대응해서 많은 쪽의 데이터를
  # 얼마나 샘플링할지 지정
  perc.under=200
)
#SMOTE 파라미터 설명 http://glemaitre.github.io/imbalanced-learn/generated/imblearn.over_sampling.SMOTE.html

### 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=500)#multi_class = 'multinomial')
log_reg.fit(X_train, y_train)

y_pred_logis = log_reg.predict(X_test)

In [ ]:
#scaling
#참고사이트 : https://mizykk.tistory.com/101

from sklearn.preprocessing import MinMaxScaler

# Normalization 최소값 0 / 최대값 1
#scaler = MinMaxScaler()

# 교차검증시
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

In [ ]:
?LogisticRegression

In [ ]:

accuracy_score(y_test, y_pred_logis)

0.6794084821428571

In [ ]:
y_test

In [ ]:

print('\n', classification_report(y_test, y_pred_logis))



               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.74      0.18      0.28       159
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        13
           4       0.71      0.81      0.76      1643
           5       0.00      0.00      0.00       111
           6       0.50      0.04      0.07        84
           7       0.00      0.00      0.00       104
           8       0.64      0.79      0.71      2065
           9       0.00      0.00      0.00        99
          10       0.67      0.07      0.13       135
          11       0.00      0.00      0.00        48
          12       0.69      0.78      0.73      1917
          13       0.00      0.00      0.00        45
          14       0.00      0.00      0.00        26
          15       0.69      0.56      0.62       670
          17       0.00      0.00      0.00        12
          18       1.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_test.value_counts().sort_values(ascending = False)

8     2065
12    1917
4     1643
15     670
1      159
10     135
5      111
7      104
9       99
6       84
11      48
13      45
18      29
14      26
3       13
17      12
2        7
0        1
Name: size, dtype: int64

In [ ]:
df_y_train_res.value_counts().sort_values(ascending = False)

size
0       8293
1       8293
2       8293
3       8293
4       8293
5       8293
6       8293
7       8293
8       8293
9       8293
10      8293
11      8293
12      8293
13      8293
14      8293
15      8293
17      8293
18      8293
dtype: int64

### 랜덤 포레스트 적용

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()

model = RandomForestClassifier(n_estimators=500,
                               min_samples_split=10, 
                               n_jobs=-1)

model

RandomForestClassifier(min_samples_split=10, n_estimators=500, n_jobs=-1)

In [ ]:
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=500, n_jobs=-1)

In [ ]:
y_pred_random = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_pred_random) * 100
score

73.28404017857143

In [ ]:

print('\n', classification_report(y_test, y_pred_random))


               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.75      0.38      0.51       159
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        13
           4       0.77      0.84      0.80      1643
           5       0.88      0.14      0.23       111
           6       0.75      0.14      0.24        84
           7       0.89      0.23      0.37       104
           8       0.71      0.81      0.76      2065
           9       0.92      0.12      0.21        99
          10       0.69      0.20      0.31       135
          11       0.57      0.08      0.15        48
          12       0.73      0.83      0.78      1917
          13       1.00      0.07      0.12        45
          14       0.00      0.00      0.00        26
          15       0.71      0.66      0.69       670
          17       0.00      0.00      0.00        12
          18       1.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_test.value_counts().sort_values(ascending = False)

8     2065
12    1917
4     1643
15     670
1      159
10     135
5      111
7      104
9       99
6       84
11      48
13      45
18      29
14      26
3       13
17      12
2        7
0        1
Name: size, dtype: int64

### 학습결과 결정경계 표기

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt


def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # 마커와 컬러맵을 설정합니다.
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # 결정 경계를 그립니다.
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # 테스트 샘플을 부각하여 그립니다.
    if test_idx:
        # 모든 샘플을 그립니다.
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    facecolor='none',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')

In [ ]:
X_combined_std = np.vstack((X_train_std, X_test_std))
y_combined = np.hstack((y_train, y_test))

plot_decision_regions(X=X_combined_std, y=y_combined,
                      classifier=ppn, test_idx=range(105, 150))
plt.xlabel('petal length [standardized]')
plt.ylabel('petal width [standardized]')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:

print('\n', classification_report(y_test, y_pred_random))


               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.74      0.37      0.49       159
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        13
           4       0.77      0.84      0.80      1643
           5       0.93      0.13      0.22       111
           6       0.76      0.15      0.26        84
           7       0.86      0.23      0.36       104
           8       0.71      0.81      0.76      2065
           9       0.92      0.12      0.21        99
          10       0.69      0.19      0.29       135
          11       0.75      0.06      0.12        48
          12       0.73      0.83      0.78      1917
          13       1.00      0.07      0.12        45
          14       0.00      0.00      0.00        26
          15       0.71      0.66      0.68       670
          17       0.00      0.00      0.00        12
          18       1.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 서포트 벡터 머신
#비선형 SVM은??

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train, y_train)

#plot_decision_regions(X_combined_std, 
#                      y_combined,
#                      classifier=svm, 
#                      test_idx=range(105, 150))

SVC(kernel='linear', random_state=1)

In [ ]:
y_pred_svc = svm.predict(X_test)


score_svc = accuracy_score(y_test, y_pred_svc) * 100
score_svc


print('\n', classification_report(y_test, y_pred_svc))


               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00       159
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        13
           4       0.70      0.82      0.76      1643
           5       0.00      0.00      0.00       111
           6       0.00      0.00      0.00        84
           7       0.00      0.00      0.00       104
           8       0.65      0.78      0.71      2065
           9       0.00      0.00      0.00        99
          10       0.00      0.00      0.00       135
          11       0.00      0.00      0.00        48
          12       0.69      0.79      0.74      1917
          13       0.00      0.00      0.00        45
          14       0.00      0.00      0.00        26
          15       0.69      0.58      0.63       670
          17       0.00      0.00      0.00        12
          18       0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 결정트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(criterion='gini', 
                                    max_depth=9, 
                                    random_state=1)
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=9, random_state=1)

In [ ]:
y_pred_tree = tree_model.predict(X_test)


score_tree = accuracy_score(y_test, y_pred_tree) * 100
score_tree


print('\n', classification_report(y_test, y_pred_tree))

72.36328125

### 퍼셉트론

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
#ppn = Perceptron(max_iter=40, eta0=0.1, tol=1e-3, random_state=1) 
ppn = Perceptron()
ppn.fit(X_train, y_train)

Perceptron()

In [ ]:
y_pred_ppn = ppn.predict(X_test)

score_ppn = accuracy_score(y_test, y_pred_ppn) * 100
score_ppn


print('\n', classification_report(y_test, y_pred_ppn))


               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00       159
           2       0.00      0.00      0.00         7
           3       0.00      0.00      0.00        13
           4       0.55      0.81      0.65      1643
           5       0.03      0.06      0.04       111
           6       0.00      0.00      0.00        84
           7       0.03      0.03      0.03       104
           8       0.44      0.27      0.34      2065
           9       0.00      0.00      0.00        99
          10       0.33      0.01      0.01       135
          11       0.04      0.25      0.07        48
          12       0.53      0.56      0.55      1917
          13       0.00      0.00      0.00        45
          14       0.00      0.00      0.00        26
          15       0.51      0.22      0.31       670
          17       0.00      0.00      0.00        12
          18       0.04  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 딥러닝 관련 import

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense # 은닉층 하나씩 정의
from tensorflow.keras.optimizers import SGD # 역전파시 사용하는 최적화 알고리즘
from tensorflow.keras.losses import mse # y가 수치형인 경우, 성능지표

tf.random.set_seed(1234) #random seed 설정

### ANN

In [ ]:
# 인공신경망 모형 구성(퍼셉트론)
model = Sequential()
model.add(Dense(4, input_shape = (13, ), activation = 'linear')) #Dense 함수가 은닉층을 표현, input shape 는 은닉층의 입력, x 변수가 두개임을 선언, 첫번째 은닉층은 노드가 3개
model.add(Dense(2, activation = 'linear')) #앞에서 이미 은닉층 입력을 지정했으므로 두번째 은닉층은 이전의 input shape x 변수는 2개 동일함
model.add(Dense(1, activation = 'linear')) #앞에서 이미 은닉층 입력을 지정했으므로 두번째 은닉층은 이전의 input shape x 변수는 2개 동일함

#모형 컴파일
model.compile(optimizer = 'SGD', loss = mse, metrics = ['acc']) #평가지표 설정
#모형 학습 및 가중치 확인
model.fit(X_train,y_train,epochs = 10)
model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)


Epoch 1/10
896/896 [==============================] - 3s 2ms/step - loss: 4.8035 - acc: 0.0279
Epoch 2/10
896/896 [==============================] - 2s 2ms/step - loss: 4.1395 - acc: 0.0280
Epoch 3/10
896/896 [==============================] - 1s 2ms/step - loss: 4.0990 - acc: 0.0280
Epoch 4/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0873 - acc: 0.0280
Epoch 5/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0761 - acc: 0.0279
Epoch 6/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0821 - acc: 0.0280
Epoch 7/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0720 - acc: 0.0280
Epoch 8/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0383 - acc: 0.0280
Epoch 9/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0420 - acc: 0.0280
Epoch 10/10
896/896 [==============================] - 1s 1ms/step - loss: 4.0417 - acc: 0.0280


[array([[ 0.09515595, -0.15785068,  0.2927242 ,  0.19948037],
        [-0.3130962 , -0.210717  ,  0.2742957 , -0.38431007],
        [ 0.0550359 ,  0.1450967 ,  0.53900254,  0.10460538],
        [-0.19605368, -0.12739864,  0.2948675 , -0.30061027],
        [-0.87825185,  0.23266393,  1.2126726 , -0.36638924],
        [-0.13261151, -0.23254238,  0.14286985, -0.05024193],
        [-0.11856927,  0.05006108,  0.11007696, -0.22091284],
        [ 0.28775868, -0.35263714,  0.2571485 , -0.29493466],
        [-0.21059862, -0.035082  ,  0.17928377, -0.23786084],
        [-0.01230456,  0.05220146,  0.06849255, -0.32166082],
        [ 0.16143246, -0.27261963,  0.22917427,  0.05175973],
        [-0.18853676, -0.3709466 ,  0.14753501,  0.1545451 ],
        [ 0.00654675,  0.07335532, -0.06558601,  0.05718767]],
       dtype=float32),
 array([-0.6060004 ,  0.42394677,  0.7516385 , -0.063312  ], dtype=float32),
 array([[ 0.42201507,  0.2199035 ],
        [-0.03214638, -0.14772089],
        [-0.6961151 ,

### DNN

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cate = to_categorical(y_train)
y_test_cate = to_categorical(y_test)



In [ ]:
# 인공신경망 모형 구성(퍼셉트론)

model = Sequential()
model.add(Dense(256, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(128, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(64, activation = 'relu' ) ) 
model.add(Dense(32, activation = 'relu' ) )
model.add(Dense(19, activation = 'softmax')) 


#모형 컴파일
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
#모형 학습 및 가중치 확인
history = model.fit(X_train, y_train_cate, epochs = 3000)#,
                    #batch_size = 128, validation_data = (x_val,y_val))

#model.fit(X_train,y_train,epochs = 10)
#model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
896/896 [==============================] - 2s 3ms/step - loss: 0.2637 - acc: 0.8993
Epoch 502/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2605 - acc: 0.9003
Epoch 503/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2410 - acc: 0.9075
Epoch 504/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2605 - acc: 0.8998
Epoch 505/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2543 - acc: 0.9041
Epoch 506/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2580 - acc: 0.9006
Epoch 507/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2835 - acc: 0.8941
Epoch 508/3000
896/896 [==============================] - 3s 3ms/step - loss: 0.2448 - acc: 0.9075
Epoch 509/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2511 - acc: 0.9038
Epoch 510/3000
896/896 [==============================] - 2s 3ms/step - loss: 0.2467 - a

In [ ]:
# 인공신경망 모형 구성(퍼셉트론)

model = Sequential()
model.add(Dense(256, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(128, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(64, activation = 'relu' ) ) 
model.add(Dense(32, activation = 'relu' ) )
model.add(Dense(19, activation = 'softmax')) 


#모형 컴파일
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
#모형 학습 및 가중치 확인
history = model.fit(X_train, y_train_cate, epochs = 3000)#,
                    #batch_size = 128, validation_data = (x_val,y_val))

#model.fit(X_train,y_train,epochs = 10)
#model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)

In [ ]:

# 인공신경망 모형 구성(퍼셉트론)

model = Sequential()
model.add(Dense(256, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(128, activation = 'relu', input_shape = (13, ) ) ) 
model.add(Dense(64, activation = 'relu' ) ) 
model.add(Dense(32, activation = 'relu' ) )
model.add(Dense(19, activation = 'softmax')) 


#모형 컴파일
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
#모형 학습 및 가중치 확인
history = model.fit(X_train, y_train_cate, epochs = 800)#,
                    #batch_size = 128, validation_data = (x_val,y_val))

#model.fit(X_train,y_train,epochs = 10)
#model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

results_DNN = model.predict( X_test)
#혼동행렬을 만듬.
#plt.figure(figsize = (7,7))
#cm = confusion_matrix(np.argmax(y_test, axis = -1), np.argmax(results, axis = -1))
#sns.heatmap(cm, annot = True, fmt = 'd')
#plt.xlabel('predicted')
#plt.ylabel('actual')
#plt.show() 

#분류 보고서
print('\n', classification_report(np.argmax(y_test_cate, axis = -1), np.argmax(results_DNN, axis = -1)))


Epoch 1/800
896/896 [==============================] - 3s 3ms/step - loss: 1.1270 - acc: 0.6444
Epoch 2/800
896/896 [==============================] - 2s 3ms/step - loss: 0.9989 - acc: 0.6765
Epoch 3/800
896/896 [==============================] - 2s 3ms/step - loss: 0.9714 - acc: 0.6801
Epoch 4/800
896/896 [==============================] - 2s 3ms/step - loss: 0.9593 - acc: 0.6829
Epoch 5/800
896/896 [==============================] - 3s 3ms/step - loss: 0.9469 - acc: 0.6860
Epoch 6/800
896/896 [==============================] - 2s 3ms/step - loss: 0.9382 - acc: 0.6884
Epoch 7/800
896/896 [==============================] - 2s 2ms/step - loss: 0.9304 - acc: 0.6880
Epoch 8/800
896/896 [==============================] - 2s 2ms/step - loss: 0.9226 - acc: 0.6914
Epoch 9/800
896/896 [==============================] - 2s 2ms/step - loss: 0.9165 - acc: 0.6934
Epoch 10/800
896/896 [==============================] - 2s 2ms/step - loss: 0.9082 - acc: 0.6941
Epoch 11/800
896/896 [=================

### 통합버전 전처리 데이터 불러오기

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/GitHub/AI-ExpertFW-Prj/dataset')

mc_rr_df_read = pd.read_excel('modcolth_rentway_integra_data3_imputed.xlsx')

In [ ]:
mc_rr_df_read_cp = mc_rr_df_read.copy()

In [ ]:
mc_rr_df_read_cp.replace({'category':['dresses', 'dress', 'shirtdress']}, 'dresses', inplace = True)
mc_rr_df_read_cp.replace({'category':['tops', 'top']}, 'tops', inplace = True)
mc_rr_df_read_cp.replace({'category':['bottoms', 'sweatpants', 'jeans', 'down']}, 'bottoms', inplace = True)
mc_rr_df_read_cp.replace({'category':['leggings', 'legging']}, 'leggings', inplace = True)
mc_rr_df_read_cp.replace({'category':['shirt', 't-shirt','sweatershirt', 'sweatshirt']}, 'shirts', inplace = True)
mc_rr_df_read_cp.replace({'category':['skirt', 'skirts']}, 'skirts', inplace = True)
mc_rr_df_read_cp.replace({'category':['coat', 'peacoat', 'overcoat']}, 'coats', inplace = True)

mc_rr_df_read_cp['rating'] = mc_rr_df_read_cp['rating']/2

In [ ]:
mc_rr_df_read_cp['rating'].value_counts()

5.0    150534
4.0     77384
3.0     28163
2.0      7352
1.0      2245
Name: rating, dtype: int64

In [ ]:
mc_rr_df_read_cp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265678 entries, 0 to 265677
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   item_id          265678 non-null  int64  
 1   size             265678 non-null  int64  
 2   rating           265678 non-null  float64
 3   category         265678 non-null  object 
 4   user_name        74542 non-null   object 
 5   length           74518 non-null   object 
 6   fit              265678 non-null  object 
 7   user_id          265678 non-null  int64  
 8   review_summary   265678 non-null  object 
 9   review_text      265678 non-null  object 
 10  waist            74542 non-null   float64
 11  hips             74542 non-null   float64
 12  bust_size        265678 non-null  float64
 13  height           265678 non-null  float64
 14  cup_size_in_cms  265678 non-null  float64
 15  rented_for       191136 non-null  object 
 16  body_type        176687 non-null  obje

In [ ]:
mc_rr_df_read_cp.describe()

,item_id,size,rating,user_id,waist,hips,bust_size,height,cup_size_in_cms,age,weight
count,2.656780e+05,265678.000000,265678.000000,265678.000000,74542.000000,74542.000000,265678.000000,265678.000000,265678.000000,190192.000000,191136.000000
mean,8.918434e+05,12.333829,4.379903,499548.980751,31.752537,40.395108,34.702768,165.768246,16.761331,33.875026,63.053087
std,7.334795e+05,8.423582,0.845591,288270.720136,4.617340,5.381014,2.345771,6.809175,2.550585,8.059234,10.057498
min,1.233730e+05,0.000000,1.000000,6.000000,20.000000,30.000000,28.000000,142.240000,10.500000,0.000000,22.700000
25%,2.691720e+05,8.000000,4.000000,251636.000000,28.231789,36.152672,34.000000,160.020000,14.500000,29.000000,56.750000
50%,6.617750e+05,12.000000,5.000000,499004.000000,31.000000,39.729514,34.000000,165.100000,16.500000,32.000000,61.290000
75%,1.382388e+06,16.000000,5.000000,749324.000000,34.000000,43.000000,36.000000,170.180000,18.500000,37.000000,68.100000
max,2.966087e+06,58.000000,5.000000,999997.000000,50.000000,60.000000,48.000000,187.960000,32.500000,117.000000,136.200000


In [ ]:
mc_rr_df_read_cp2.describe()

,item_id,size,rating,user_id,waist,hips,bust_size,height,cup_size_in_cms,weight,age
count,2.656780e+05,265678.000000,265678.000000,265678.000000,74542.000000,74542.000000,265678.000000,265678.000000,265678.000000,265678.000000,265678.000000
mean,8.918434e+05,12.333829,4.379903,499548.980751,31.752537,40.395108,34.702768,165.768246,16.761331,65.526956,34.201773
std,7.334795e+05,8.423582,0.845591,288270.720136,4.617340,5.381014,2.345771,6.809175,2.550585,12.251326,7.104792
min,1.233730e+05,0.000000,1.000000,6.000000,20.000000,30.000000,28.000000,142.240000,10.500000,22.700000,0.000000
25%,2.691720e+05,8.000000,4.000000,251636.000000,28.231789,36.152672,34.000000,160.020000,14.500000,56.750000,30.000000
50%,6.617750e+05,12.000000,5.000000,499004.000000,31.000000,39.729514,34.000000,165.100000,16.500000,62.882534,33.000000
75%,1.382388e+06,16.000000,5.000000,749324.000000,34.000000,43.000000,36.000000,170.180000,18.500000,70.733200,37.100000
max,2.966087e+06,58.000000,5.000000,999997.000000,50.000000,60.000000,48.000000,187.960000,32.500000,136.200000,117.000000


### 통합버전 데이터 전처리

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
mc_data_knn_ind_features = mc_rr_df_read_cp[['bust_size','height', 'cup_size_in_cms', 'weight', 'age']]

df_filled = imputer.fit_transform(mc_data_knn_ind_features)

knn_numeric_imputations = pd.DataFrame(data=df_filled, columns=['bust_size','height', 'cup_size_in_cms', 'weight', 'age'],index = mc_rr_df_read_cp.index)

mc_rr_df_read_cp2 = mc_rr_df_read_cp.drop(['bust_size','height', 'cup_size_in_cms', 'weight', 'age'], axis=1)

mc_rr_df_read_cp2 = pd.concat([mc_rr_df_read_cp2, knn_numeric_imputations], axis=1)

#mc_rr_df_read_cp2.to_excel('modcolth_rentway_integra_data3_imputed2.xlsx', index = False)
mc_rr_df_read_cp2 = pd.read_excel('modcolth_rentway_integra_data3_imputed2.xlsx')
#27분 소요



In [ ]:
mc_rr_df_read_cp2.head()

,item_id,size,rating,fit,user_id,review_summary,review_text,rented_for,body_type,bust_size,...,cate_tee,cate_tight,cate_tops,cate_trench,cate_trouser,cate_trousers,cate_tunic,cate_turtleneck,cate_vest,cate_wedding
0,152702,4,4,1,668176,Too much ruching,"I liked the color, the silhouette, and the fab...",NaN,NaN,32.0,...,0,0,0,0,0,0,0,0,0,0
1,152702,4,5,2,320759,Suits my body type!,From the other reviews it seems like this dres...,NaN,NaN,34.0,...,0,0,0,0,0,0,0,0,0,0
2,152702,4,3,2,144785,I love the design and fit,I love the design and fit of this dress! I wo...,NaN,NaN,34.0,...,0,0,0,0,0,0,0,0,0,0
3,152702,4,3,2,52664,Beautiful Dress!,I bought this dress for work it is flattering...,NaN,NaN,32.0,...,0,0,0,0,0,0,0,0,0,0
4,152702,4,5,2,155439,This is a very profession,This is a very professional look. It is Great ...,NaN,NaN,32.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
mc_rr_df_read_cp

,item_id,size,rating,category,user_name,length,fit,user_id,review_summary,review_text,waist,hips,bust_size,height,cup_size_in_cms,rented_for,body_type,age,review_date,weight
0,152702,4,4.0,new,avNYC,just right,small,668176,Too much ruching,"I liked the color, the silhouette, and the fab...",27.0,37.000000,32.0,167.64,14.5,NaN,NaN,NaN,NaN,NaN
1,152702,4,5.0,new,lanwei91,slightly short,fit,320759,Suits my body type!,From the other reviews it seems like this dres...,26.0,36.000000,34.0,167.64,16.5,NaN,NaN,NaN,NaN,NaN
2,152702,4,3.0,new,angeladevoe5678,just right,fit,144785,I love the design and fit,I love the design and fit of this dress! I wo...,25.0,35.765625,34.0,160.02,12.5,NaN,NaN,NaN,NaN,NaN
3,152702,4,3.0,new,Juli,slightly long,fit,52664,Beautiful Dress!,I bought this dress for work it is flattering...,25.0,35.000000,32.0,154.94,16.5,NaN,NaN,NaN,NaN,NaN
4,152702,4,5.0,new,lhalton,just right,fit,155439,This is a very profession,This is a very professional look. It is Great ...,25.0,32.000000,32.0,167.64,16.5,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265673,2252812,8,5.0,jumpsuit,NaN,NaN,fit,66386,LOVE IT!!! First Item Im thinking of buying!,Fit like a glove!,NaN,NaN,34.0,175.26,20.5,work,hourglass,42.0,"May 18, 2016",63.56
265674,682043,4,5.0,dresses,NaN,NaN,fit,118398,LOVE it!,The pattern contrast on this dress is really s...,NaN,NaN,32.0,154.94,16.5,work,petite,29.0,"September 30, 2016",45.40
265675,683251,8,3.0,dresses,NaN,NaN,fit,47002,"Loud patterning, flattering fit","Like the other DVF wraps, the fit on this is f...",NaN,NaN,36.0,172.72,12.5,everyday,straight & narrow,31.0,"March 4, 2016",61.29
265676,126335,16,5.0,dresses,NaN,NaN,fit,961120,loved this dress it was comfortable and photog...,This dress was PERFECTION. it looked incredib...,NaN,NaN,36.0,167.64,16.5,wedding,pear,31.0,"November 25, 2015",74.91


In [ ]:
# apply one hot encoding using dummies

#length_dummies  = pd.get_dummies(modcloth_preprocessing2['length'])

category_dummies  = pd.get_dummies(mc_rr_df_read_cp2['category'], prefix='cate')

#modcloth_preprocessing = pd.concat([modcloth_preprocessing, length_dummies], axis = 1)
mc_rr_df_read_cp2 = pd.concat([mc_rr_df_read_cp2,category_dummies], axis = 1)

#modcloth_preprocessing.drop(['length'], axis=1, inplace=True)
mc_rr_df_read_cp2.drop(['category'], axis=1, inplace=True)

# target variable 
fit = {'small':1, 'fit':2, 'large':3}
mc_rr_df_read_cp2['fit'] = mc_rr_df_read_cp2['fit'].map(fit)

#length = {'very short' : 1, 'slightly short' : 2, 'just right' : 3, 'slightly long' : 4, 'very long' : 5}
#modcloth_preprocessing2['length'] = modcloth_preprocessing2['length'].map(length)


In [ ]:
mc_rr_df_read_cp2.drop(['user_name', 'length', 'waist', 'hips', 'review_date'], axis=1, inplace=True)

In [ ]:
mc_rr_df_read_cp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265678 entries, 0 to 265677
Data columns (total 76 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   item_id          265678 non-null  int64  
 1   size             265678 non-null  int64  
 2   rating           265678 non-null  float64
 3   fit              265678 non-null  int64  
 4   user_id          265678 non-null  int64  
 5   review_summary   265678 non-null  object 
 6   review_text      265678 non-null  object 
 7   rented_for       191136 non-null  object 
 8   body_type        176687 non-null  object 
 9   bust_size        265678 non-null  float64
 10  height           265678 non-null  float64
 11  cup_size_in_cms  265678 non-null  float64
 12  weight           265678 non-null  float64
 13  age              265678 non-null  float64
 14  cate_ballgown    265678 non-null  uint8  
 15  cate_blazer      265678 non-null  uint8  
 16  cate_blouse      265678 non-null  uint

In [ ]:
#X_data = mc_rr_df_read_cp2[(mc_rr_df_read_cp2['fit'] == 2) ][['bust_size',	'height',	'cup_size_in_cms',	'weight',	'age',	'cate_ballgown',	'cate_blazer',	'cate_blouse',	'cate_blouson',	'cate_bomber',	'cate_bottoms',	'cate_buttondown',	'cate_caftan',	'cate_cami',	'cate_cape',	'cate_cardigan',	'cate_coats',	'cate_combo',	'cate_crewneck',	'cate_culotte',	'cate_culottes',	'cate_dresses',	'cate_duster',	'cate_for',	'cate_frock',	'cate_gown',	'cate_henley',	'cate_hoodie',	'cate_jacket',	'cate_jogger',	'cate_jumpsuit',	'cate_kaftan',	'cate_kimono',	'cate_knit',	'cate_leggings',	'cate_maxi',	'cate_midi',	'cate_mini',	'cate_new',	'cate_outerwear',	'cate_overalls',	'cate_pant',	'cate_pants',	'cate_parka',	'cate_poncho',	'cate_print',	'cate_pullover',	'cate_romper',	'cate_sale',	'cate_sheath',	'cate_shift',	'cate_shirts',	'cate_skirts',	'cate_skort',	'cate_suit',	'cate_sweater',	'cate_tank',	'cate_tee',	'cate_tight',	'cate_tops',	'cate_trench',	'cate_trouser',	'cate_trousers',	'cate_tunic',	'cate_turtleneck',	'cate_vest',	'cate_wedding']]

X_data = mc_rr_df_read_cp2[(mc_rr_df_read_cp2['fit'] == 2) ][['bust_size',	'height',	'cup_size_in_cms',	'weight',	'age']]
Y_data = mc_rr_df_read_cp2[(mc_rr_df_read_cp2['fit'] == 2) ]['size']


In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2 # taking 70:30 training and test set
seed = 1  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=test_size, random_state=seed)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 

# 교차검증시
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


### 로지스틱 회귀

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=300)#multi_class = 'multinomial')
log_reg.fit(X_train, y_train)

y_pred_logis = log_reg.predict(X_test)
#7분소요

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
accuracy_score(y_test, y_pred_logis)

0.3759781619654231

In [ ]:
print('\n', classification_report(y_test, y_pred_logis))


               precision    recall  f1-score   support

           0       0.00      0.00      0.00        89
           1       0.56      0.37      0.45      2357
           2       0.44      0.05      0.09       134
           3       0.00      0.00      0.00       127
           4       0.48      0.53      0.50      6103
           5       0.34      0.04      0.08       342
           6       0.00      0.00      0.00       114
           7       0.00      0.00      0.00       210
           8       0.41      0.64      0.50      8558
           9       0.28      0.16      0.21       525
          10       0.00      0.00      0.00       177
          11       0.00      0.00      0.00       138
          12       0.29      0.36      0.32      5806
          13       0.25      0.25      0.25       457
          14       0.41      0.08      0.14      1735
          15       0.26      0.10      0.14       846
          16       0.29      0.03      0.05      2471
          17       0.27  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_test.value_counts().sort_values(ascending = False)

8     8558
4     6103
12    5806
20    3360
16    2471
1     2357
14    1735
24    1363
15     846
26     740
32     559
9      525
28     469
13     457
17     371
5      342
21     278
7      210
25     189
39     177
10     177
38     163
11     138
2      134
3      127
45     126
35     126
6      114
0       89
51      73
29      66
36      35
57      32
18      27
23      18
46      15
27      15
19      12
40      11
33       9
48       9
42       7
43       6
30       5
52       4
22       3
44       3
54       2
50       1
34       1
56       1
Name: size, dtype: int64

### 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()

model = RandomForestClassifier(n_estimators=300,
                               min_samples_split=10, 
                               n_jobs=-1)

model

RandomForestClassifier(min_samples_split=10, n_estimators=300, n_jobs=-1)

In [ ]:
model.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=300, n_jobs=-1)

In [ ]:
y_pred_random = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, y_pred_random) * 100
score

42.313791758741715

In [ ]:

print('\n', classification_report(y_test, y_pred_random))


               precision    recall  f1-score   support

           0       0.27      0.03      0.06        89
           1       0.54      0.43      0.48      2357
           2       0.48      0.16      0.24       134
           3       0.20      0.01      0.02       127
           4       0.49      0.55      0.52      6103
           5       0.34      0.18      0.24       342
           6       0.32      0.05      0.09       114
           7       0.00      0.00      0.00       210
           8       0.47      0.56      0.51      8558
           9       0.26      0.18      0.21       525
          10       0.00      0.00      0.00       177
          11       0.00      0.00      0.00       138
          12       0.39      0.43      0.41      5806
          13       0.26      0.23      0.24       457
          14       0.33      0.17      0.22      1735
          15       0.29      0.21      0.25       846
          16       0.35      0.33      0.34      2471
          17       0.27  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1

### SVM

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train, y_train)

#plot_decision_regions(X_combined_std, 
#                      y_combined,
#                      classifier=svm, 
#                      test_idx=range(105, 150))

In [ ]:
y_pred_svc = svm.predict(X_test)


score_svc = accuracy_score(y_test, y_pred_svc) * 100
score_svc


print('\n', classification_report(y_test, y_pred_svc))

### 결정트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(criterion='gini', 
                                    max_depth=9, 
                                    random_state=1)
tree_model.fit(X_train, y_train)

In [ ]:
y_pred_tree = tree_model.predict(X_test)


score_tree = accuracy_score(y_test, y_pred_tree) * 100
score_tree


print('\n', classification_report(y_test, y_pred_tree))

### 퍼셉트론

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
#ppn = Perceptron(max_iter=40, eta0=0.1, tol=1e-3, random_state=1) 
ppn = Perceptron()
ppn.fit(X_train, y_train)

In [ ]:
y_pred_ppn = ppn.predict(X_test)

score_ppn = accuracy_score(y_test, y_pred_ppn) * 100
score_ppn


print('\n', classification_report(y_test, y_pred_ppn))

### 딥러닝 관련 import

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense # 은닉층 하나씩 정의
from tensorflow.keras.optimizers import SGD # 역전파시 사용하는 최적화 알고리즘
from tensorflow.keras.losses import mse # y가 수치형인 경우, 성능지표

tf.random.set_seed(1234) #random seed 설정

### ANN

In [ ]:
# 인공신경망 모형 구성(퍼셉트론)
model = Sequential()
model.add(Dense(4, input_shape = (67, ), activation = 'linear')) #Dense 함수가 은닉층을 표현, input shape 는 은닉층의 입력, x 변수가 두개임을 선언, 첫번째 은닉층은 노드가 3개
model.add(Dense(2, activation = 'linear')) #앞에서 이미 은닉층 입력을 지정했으므로 두번째 은닉층은 이전의 input shape x 변수는 2개 동일함
model.add(Dense(1, activation = 'linear')) #앞에서 이미 은닉층 입력을 지정했으므로 두번째 은닉층은 이전의 input shape x 변수는 2개 동일함

#모형 컴파일
model.compile(optimizer = 'SGD', loss = mse, metrics = ['acc']) #평가지표 설정
#모형 학습 및 가중치 확인
model.fit(X_train,y_train,epochs = 10)
model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)


### DNN


In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cate = to_categorical(y_train)
y_test_cate = to_categorical(y_test)

In [ ]:
# 인공신경망 모형 구성(퍼셉트론)

model = Sequential()
model.add(Dense(256, activation = 'relu', input_shape = (67, ) ) ) 
model.add(Dense(128, activation = 'relu') ) 
model.add(Dense(64, activation = 'relu' ) ) 
model.add(Dense(32, activation = 'relu' ) )
model.add(Dense(19, activation = 'softmax')) 


#모형 컴파일
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])
#모형 학습 및 가중치 확인
history = model.fit(X_train, y_train_cate, epochs = 3000)#,
                    #batch_size = 128, validation_data = (x_val,y_val))

#model.fit(X_train,y_train,epochs = 10)
#model.get_weights() #인공신경망 구성하는 각각의 가중치(2*3 = 6개의 가중치, 그다음 배열은 bias, 첫번째 은닉층에서 두번째 은닉층으로의 가중치 3개, 그 다음 배열은 bias로 총 4개의 배열)